In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

In [4]:
#!pip install implicit --no-cache-dir

In [5]:
df = pd.read_csv('user_taggedbookmarks-timestamps.dat', sep='\t')
df.head()

,userID,bookmarkID,tagID,timestamp
0,8,1,1,1289255362000
1,8,2,1,1289255159000
2,8,7,1,1289238901000
3,8,7,6,1289238901000
4,8,7,7,1289238901000


In [6]:
df.shape

(437593, 4)

In [7]:
del df['timestamp']
del df['tagID']

In [8]:
df['user_id'] = df['userID'].astype("category").cat.codes
df['bookmark_id'] = df['bookmarkID'].astype("category").cat.codes

In [9]:
bookmark_lookup = df[['bookmark_id', 'bookmarkID']].drop_duplicates()
user_lookup = df[['user_id', 'userID']].drop_duplicates()

In [10]:
values = [1] * df.shape[0]
cols = df.bookmark_id.astype(int)
rows = df.user_id.astype(int)

In [11]:
data = sparse.csr_matrix((values, (cols, rows)), shape=(len(df.bookmarkID.unique()), len(df.userID.unique())))

In [13]:
model = AlternatingLeastSquares(factors=50)
model.fit(data)

In [14]:
userid = 0

user_items = data.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [19]:
recommendations

[(3270, 0.25655162),
 (19854, 0.21482605),
 (19839, 0.21056652),
 (9887, 0.20690529),
 (1137, 0.20206578),
 (4332, 0.20034556),
 (19862, 0.19951467),
 (19860, 0.19892518),
 (19866, 0.19484365),
 (19861, 0.18996581)]

In [25]:
for r in recommendations:
    print(bookmark_lookup.iloc[r[0],1])

4638
29116
29098
14148
1631
6090
29126
29124
29132
29125
